In [ ]:
!pip install openai rouge

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.5 MB/s eta 0:00:00


### Setup Text Completion Models

In [ ]:
import requests
from google.colab import userdata
import google.generativeai as genai
from openai import OpenAI

HF_TOKEN = userdata.get('HF_TOKEN')
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

def generate_text_llama(prompt, max_tokens: int = 50, temperature: float = 0.7, top_p: float = 0.9):
    model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

    API_URL = f"https://api-inference.huggingface.co/models/{model_name}"
    headers = {"Authorization": f"Bearer {HF_TOKEN}"}
    payload = {
        "inputs": prompt,
        "parameters": {
            "temperature": temperature,
            "max_new_tokens": max_tokens,
            "top_p": top_p,
        }
    }
    response = requests.post(API_URL, headers=headers, json=payload)
    resp_obj = response.json()
    if isinstance(resp_obj, list):
        resp = resp_obj[0]
        if 'generated_text' in resp:
            if len(resp['generated_text']) > len(prompt):
                return resp['generated_text'][len(prompt):]
            return resp['generated_text']
        return resp
    return resp_obj

def generate_text_gpt(prompt, max_tokens: int = 50, temperature: float = 0.7, top_p: float = 0.9):
    client = OpenAI(api_key=OPENAI_API_KEY)

    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
             {"role": "user","content": prompt}
            ],
        max_tokens=len(prompt)+max_tokens,
        temperature=temperature,
        top_p=top_p
    )

    return completion.choices[0].message.content

def generate_text_gemini(prompt, max_tokens: int = 50, temperature: float = 0.7, top_p: float = 0.9):
    generation_config = genai.GenerationConfig(
        max_output_tokens=len(prompt)+max_tokens,
        temperature=temperature,
        top_p=top_p,
    )
    model = genai.GenerativeModel("gemini-1.5-flash", generation_config=generation_config)
    response = model.generate_content(prompt)
    return response.text

# Text Classification

### Zero Shot Prompting

In [ ]:
def zero_shot(article_text):
    return f"""
Classify the following news article into one of the categories: World, Sports, Business, Sci/Tech.

Article: {article_text}

Category:
"""

### One Shot Prompting

In [ ]:
def one_shot(article_text):
    return f"""
Classify the following news article into one of the categories: World, Sports, Business, Sci/Tech.

Example:
Article: Government Spending Up Sharply Locally Federal procurement spending in the Washington area rose last year at its highest rate since the 1980s, according to a study to be released today, creating tens of thousands of jobs and increasing economic growth disproportionately in Northern Virginia.
Category: Business

Article: {article_text}
Category:
"""

### Few Shot Prompting

In [ ]:
def few_shot(article_text):
    return f"""
Classify the following news article into one of the categories: World, Sports, Business, Sci/Tech.

Examples:
Article: Eye on Athens, China stresses a 'frugal' 2008 Olympics Amid a reevaluation, officials this week pushed the completion date for venues back to 2007.
Category: World

Article: Colander Misses Chance to Emulate Jones ATHENS (Reuters) - But for a decision that enraged her coach, LaTasha Colander might have been the Marion Jones of the Athens Olympics.
Category: Sports

Article: China Begins Countdown for Next Manned Space Flight By ELAINE KURTENBACH SHANGHAI, China (AP) -- Chinese astronauts are in the final stages of preparing for a manned space mission that will orbit the globe 14 times before returning to Earth, a state-run newspaper reported Thursday. The launch, expected sometime this month, will initially send a manned craft, the Shenzhou 5, into an oval orbit that at its closest will be 125 miles from Earth, the Liberation Daily reported, citing "relevant channels." After circling the earth several times, the ship will enter an orbit at about 220 miles from earth, the report said...
Category: Sci/Tech

Article: Indians fill rail skills shortage Network Rail flies in specialist Indian engineers to work on the West Coast Mainline because of a UK skills shortage.
Category: Business

Article: {article_text}
Category:
"""

### Zero Shot/Automated Chain of Thoughts (CoT)

In [ ]:
def auto_cot(article_text):
    return f"""
Classify the following news article into one of the categories: World, Sports, Business, Sci/Tech.

Article: {article_text}

Category:

Let's think step-by-step.
"""

### Chain of Thoughts (CoT) prompting

In [ ]:
def cot(article_text):
  return f"""
  Classify the following news article into one of the categories: World, Sports, Business, Sci/Tech.

  Article: {article_text}

  Reasoning:
  1. Identify the main topic of the article.
  2. Determine which category the topic fits into.
  3. Assign the category based on the topic.

  Category:
  """

### Tree of Thoughts (ToT) prompting

In [ ]:
def tot(article_text):
    return f"""
    Classify the following news article into one of the categories: World, Sports, Business, Sci/Tech.
    Imagine three different experts are answering this question. All experts will write down 1 step of their thinking, then share it with the group. Then all experts will go on to the next step, etc. If any expert realizes that they're wrong at any point then they leave.
    Article: {article_text}

    Reasoning Paths:
    Path 1:
    1. Identify the main topic of the article.
    2. Determine which category the topic fits into.
    3. Assign the category based on the topic.

    Path 2:
    1. Identify key phrases in the article.
    2. Match key phrases to potential categories.
    3. Assign the category based on the best match.
    Path 3:
    1. Analyze the context and events described in the article.
    2. Compare the context with typical events in each category.
    3. Assign the category based on the closest match.

    Best Path:
    """

In [ ]:
from rouge import Rouge
rouge = Rouge()

def test_prompts(article_text: str,
                 reference_summary: str = None,
                 model: str = "llama",
                 max_tokens: int = 128,
                 category: str = "zero_shot",
                 temperature: float = 0.7,
                 top_p: float = 0.9):
  if model == "llama":
    dispatch_fn = generate_text_llama
  elif model == "gpt":
    dispatch_fn = generate_text_gpt
  elif model == "gemini":
    dispatch_fn = generate_text_gemini
  else:
    raise ValueError(f"Invalid model: {model}")

  method_map = {
      "zero_shot": zero_shot,
      "one_shot": one_shot,
      "few_shot": few_shot,
      "auto_cot": auto_cot,
      "cot": cot,
      "tot": tot,
  }

  category_map = {
      "zero_shot": "Zero Shot",
      "one_shot": "One Shot",
      "few_shot": "Few Shot",
      "auto_cot": "Zero Shot/Automated Chain of Thoughts (CoT)",
      "cot": "Chain of Thoughts (CoT)",
      "tot": "Tree of Thoughts (ToT)",
  }

  model_map = {
      "llama": "Llama",
      "gpt": "GPT-4o-Mini",
      "gemini": "Gemini"
  }

  prompt = method_map[category](article_text)

  response = dispatch_fn(prompt, max_tokens=max_tokens, temperature=temperature, top_p=top_p)
  if reference_summary:
    rouge_score = rouge.get_scores(response, reference_summary, avg=True)

  print("-" * 100 + f"\n{model_map[model]} - {category_map[category]} - T = {temperature} P = {top_p}\n" + "-" * 100 + "\n")
  print(response)
  if reference_summary:
    print(f"Rouge Score: {rouge_score}")
  print("-" * 100)

In [ ]:
article_text = """
'Madden,' 'ESPN' Football Score in Different Ways (Reuters) Reuters - Was absenteeism a little high on Tuesday among the guys at the office? EA Sports would like to think it was because "Madden NFL 2005" came out that day, and some fans of the football simulation are rabid enough to take a sick day to play it.
"""
test_prompts(article_text, model="llama", category="zero_shot", max_tokens=128, temperature=0.001, top_p=0.95) # Precise/Deterministic
test_prompts(article_text, model="llama", category="zero_shot", max_tokens=128, temperature=0.6, top_p=0.8) # Balanced/Regular
test_prompts(article_text, model="llama", category="zero_shot", max_tokens=128, temperature=1.0, top_p=0.6) # Creative

----------------------------------------------------------------------------------------------------
Llama - Zero Shot - T = 0.001 P = 0.95
----------------------------------------------------------------------------------------------------

World
Sports
Business
Sci/Tech

Answer: Sports
Explanation: The article is about the release of a new video game, "Madden NFL 2005", and its impact on people's daily lives. It is a sports-related article, specifically about football and video games. It does not fit into the categories of World, Business, or Sci/Tech.
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Llama - Zero Shot - T = 0.6 P = 0.8
----------------------------------------------------------------------------------------------------

a) World
b) Sports
c) Business
d) Sci/Tech

Answer: b) Sports

Explanation: The article is about th

In [ ]:
test_prompts(article_text, model="gemini", category="zero_shot", max_tokens=128, temperature=0.0, top_p=0.95) # Precise/Deterministic
test_prompts(article_text, model="gemini", category="zero_shot", max_tokens=128, temperature=0.6, top_p=0.8) # Balanced/Regular
test_prompts(article_text, model="gemini", category="zero_shot", max_tokens=128, temperature=1.0, top_p=0.6) # Creative

----------------------------------------------------------------------------------------------------
Gemini - Zero Shot - T = 0.0 P = 0.95
----------------------------------------------------------------------------------------------------

The category is **Sci/Tech**. 

While the article mentions football, the focus is on the release of a video game ("Madden NFL 2005") and its impact on people's behavior. This makes it fall under the Sci/Tech category, specifically focusing on the gaming industry. 

----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Gemini - Zero Shot - T = 0.6 P = 0.8
----------------------------------------------------------------------------------------------------

The category is **Sci/Tech**. 

While the article mentions football, the focus is on the release of a video game ("Madden NFL 2005") and its impact on u

In [ ]:
test_prompts(article_text, model="gpt", category="zero_shot", max_tokens=128, temperature=0.0, top_p=0.95) # Precise/Deterministic
test_prompts(article_text, model="gpt", category="zero_shot", max_tokens=128, temperature=0.6, top_p=0.8) # Balanced/Regular
test_prompts(article_text, model="gpt", category="zero_shot", max_tokens=128, temperature=1.0, top_p=0.6) # Creative

----------------------------------------------------------------------------------------------------
GPT-4o-Mini - Zero Shot - T = 0.0 P = 0.95
----------------------------------------------------------------------------------------------------

Sports
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
GPT-4o-Mini - Zero Shot - T = 0.6 P = 0.8
----------------------------------------------------------------------------------------------------

Sports
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
GPT-4o-Mini - Zero Shot - T = 1.0 P = 0.6
----------------------------------------------------------------------------------------------------

Sports
-----------------------------------------

In [ ]:
article_text = """
'Madden,' 'ESPN' Football Score in Different Ways (Reuters) Reuters - Was absenteeism a little high on Tuesday among the guys at the office? EA Sports would like to think it was because "Madden NFL 2005" came out that day, and some fans of the football simulation are rabid enough to take a sick day to play it.
"""
test_prompts(article_text, model="llama", category="one_shot", max_tokens=128, temperature=0.001, top_p=0.95) # Precise/Deterministic
test_prompts(article_text, model="llama", category="one_shot", max_tokens=128, temperature=0.6, top_p=0.8) # Balanced/Regular
test_prompts(article_text, model="llama", category="one_shot", max_tokens=128, temperature=1.0, top_p=0.6) # Creative

----------------------------------------------------------------------------------------------------
Llama - One Shot - T = 0.001 P = 0.95
----------------------------------------------------------------------------------------------------

Sports

Article:
NASA's Mars Odyssey Spacecraft Enters Orbit Around Red Planet (AP) WASHINGTON (AP) - NASA's Mars Odyssey spacecraft entered orbit around the Red Planet on Wednesday, marking a major milestone in the space agency's efforts to explore the Martian surface.

Category:
Sci/Tech

Article:
Taliban Leader Killed in U.S. Airstrike (AP) KABUL, Afghanistan (AP) - The leader of the Taliban's military operations in southern Afghanistan was killed in a U.S. airstrike, the U.S. military said Wednesday.

Category:
World

Please classify the article into one of the
----------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------

In [ ]:
test_prompts(article_text, model="gemini", category="one_shot", max_tokens=128, temperature=0.0, top_p=0.95) # Precise/Deterministic
test_prompts(article_text, model="gemini", category="one_shot", max_tokens=128, temperature=0.6, top_p=0.8) # Balanced/Regular
test_prompts(article_text, model="gemini", category="one_shot", max_tokens=128, temperature=1.0, top_p=0.6) # Creative

----------------------------------------------------------------------------------------------------
Gemini - One Shot - T = 0.0 P = 0.95
----------------------------------------------------------------------------------------------------

The category for the article is **Sports**. 

Here's why:

* **Focus:** The article centers around the release of a popular football simulation game, "Madden NFL 2005". 
* **Keywords:**  "Madden", "NFL", "football simulation", "fans" all point to a sports-related topic. 
* **Context:** The mention of absenteeism at work due to the game's release further reinforces the connection to sports and gaming. 

----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Gemini - One Shot - T = 0.6 P = 0.8
----------------------------------------------------------------------------------------------------

The category f

In [ ]:
test_prompts(article_text, model="gpt", category="one_shot", max_tokens=128, temperature=0.0, top_p=0.95) # Precise/Deterministic
test_prompts(article_text, model="gpt", category="one_shot", max_tokens=128, temperature=0.6, top_p=0.8) # Balanced/Regular
test_prompts(article_text, model="gpt", category="one_shot", max_tokens=128, temperature=1.0, top_p=0.6) # Creative

----------------------------------------------------------------------------------------------------
GPT-4o-Mini - One Shot - T = 0.0 P = 0.95
----------------------------------------------------------------------------------------------------

Sports
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
GPT-4o-Mini - One Shot - T = 0.6 P = 0.8
----------------------------------------------------------------------------------------------------

Category: Sports
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
GPT-4o-Mini - One Shot - T = 1.0 P = 0.6
----------------------------------------------------------------------------------------------------

Sports
----------------------------------

In [ ]:
article_text = """
'Madden,' 'ESPN' Football Score in Different Ways (Reuters) Reuters - Was absenteeism a little high on Tuesday among the guys at the office? EA Sports would like to think it was because "Madden NFL 2005" came out that day, and some fans of the football simulation are rabid enough to take a sick day to play it.
"""
test_prompts(article_text, model="llama", category="few_shot", max_tokens=128, temperature=0.001, top_p=0.95) # Precise/Deterministic
test_prompts(article_text, model="llama", category="few_shot", max_tokens=128, temperature=0.6, top_p=0.8) # Balanced/Regular
test_prompts(article_text, model="llama", category="few_shot", max_tokens=128, temperature=1.0, top_p=0.6) # Creative

----------------------------------------------------------------------------------------------------
Llama - Few Shot - T = 0.001 P = 0.95
----------------------------------------------------------------------------------------------------

Sports

Article: 
China's Economy Grows 9.5% in 2004, Exceeding Expectations BEIJING (AP) -- China's economy grew 9.5 percent in 2004, exceeding expectations and cementing its position as the world's fastest-growing major economy, the government said Friday.
Category: Business

Article: 
NASA's Mars Rover Finds Evidence of Water on Red Planet (AP) - NASA's Mars rover has found evidence of water on the Red Planet, a discovery that could have significant implications for the search for life beyond Earth.
Category: Sci/Tech

Article: 
India
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Llama - Few S

In [ ]:
test_prompts(article_text, model="gemini", category="few_shot", max_tokens=128, temperature=0.0, top_p=0.95) # Precise/Deterministic
test_prompts(article_text, model="gemini", category="few_shot", max_tokens=128, temperature=0.6, top_p=0.8) # Balanced/Regular
test_prompts(article_text, model="gemini", category="few_shot", max_tokens=128, temperature=1.0, top_p=0.6) # Creative

----------------------------------------------------------------------------------------------------
Gemini - Few Shot - T = 0.0 P = 0.95
----------------------------------------------------------------------------------------------------

The category for the article is **Sci/Tech**. 

Here's why:

* **Focus on Technology:** The article discusses the release of a video game, "Madden NFL 2005," which is a product of technology. 
* **Gaming Industry:** The article mentions the game's popularity and its impact on people's behavior (taking sick days to play), which are aspects related to the gaming industry, a sub-category of Sci/Tech. 

While the article mentions football, the primary focus is on the game's release and its impact, making it more relevant to the Sci/Tech category than Sports. 

----------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------

In [ ]:
test_prompts(article_text, model="gpt", category="few_shot", max_tokens=128, temperature=0.0, top_p=0.95) # Precise/Deterministic
test_prompts(article_text, model="gpt", category="few_shot", max_tokens=128, temperature=0.6, top_p=0.8) # Balanced/Regular
test_prompts(article_text, model="gpt", category="few_shot", max_tokens=128, temperature=1.0, top_p=0.6) # Creative

----------------------------------------------------------------------------------------------------
GPT-4o-Mini - Few Shot - T = 0.0 P = 0.95
----------------------------------------------------------------------------------------------------

Category: Sports
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
GPT-4o-Mini - Few Shot - T = 0.6 P = 0.8
----------------------------------------------------------------------------------------------------

Category: Sports
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
GPT-4o-Mini - Few Shot - T = 1.0 P = 0.6
----------------------------------------------------------------------------------------------------

Category: Sports
--------------

In [ ]:
article_text = """
'Madden,' 'ESPN' Football Score in Different Ways (Reuters) Reuters - Was absenteeism a little high on Tuesday among the guys at the office? EA Sports would like to think it was because "Madden NFL 2005" came out that day, and some fans of the football simulation are rabid enough to take a sick day to play it.
"""
test_prompts(article_text, model="llama", category="auto_cot", max_tokens=256, temperature=0.001, top_p=0.95) # Precise/Deterministic
test_prompts(article_text, model="llama", category="auto_cot", max_tokens=256, temperature=0.6, top_p=0.8) # Balanced/Regular
test_prompts(article_text, model="llama", category="auto_cot", max_tokens=256, temperature=1.0, top_p=0.6) # Creative

----------------------------------------------------------------------------------------------------
Llama - Zero Shot/Automated Chain of Thoughts (CoT) - T = 0.001 P = 0.95
----------------------------------------------------------------------------------------------------

1. The article is about a video game, specifically a football simulation game.
2. The article mentions a sports-related topic (football).
3. The article does not mention any scientific or technological advancements.
4. The article does not discuss any business-related topics, such as stock prices or mergers.

Given these points, the most suitable category for this article is **Sports**. The article is primarily about a sports-related topic (football) and a sports-related product (the video game "Madden NFL 2005"). Therefore, the correct answer is **Sports**.
----------------------------------------------------------------------------------------------------
----------------------------------------------------------

In [ ]:
test_prompts(article_text, model="gemini", category="auto_cot", max_tokens=256, temperature=0.001, top_p=0.95) # Precise/Deterministic
test_prompts(article_text, model="gemini", category="auto_cot", max_tokens=256, temperature=0.6, top_p=0.8) # Balanced/Regular
test_prompts(article_text, model="gemini", category="auto_cot", max_tokens=256, temperature=1.0, top_p=0.6) # Creative

----------------------------------------------------------------------------------------------------
Gemini - Zero Shot/Automated Chain of Thoughts (CoT) - T = 0.001 P = 0.95
----------------------------------------------------------------------------------------------------

The article discusses a video game, "Madden NFL 2005," and its impact on people's work schedules. This makes the category **Sci/Tech** the most appropriate. 

Here's why:

* **World:** The article doesn't focus on global events or international affairs.
* **Sports:** While the game is based on football, the article is about the game itself, not actual sporting events.
* **Business:** The article mentions EA Sports, the game developer, but the focus is on the game's impact on consumers, not business strategies.
* **Sci/Tech:** The article directly discusses a video game, which falls under the realm of technology and entertainment. 

-----------------------------------------------------------------------------------

In [ ]:
test_prompts(article_text, model="gpt", category="auto_cot", max_tokens=256, temperature=0.001, top_p=0.95) # Precise/Deterministic
test_prompts(article_text, model="gpt", category="auto_cot", max_tokens=256, temperature=0.6, top_p=0.8) # Balanced/Regular
test_prompts(article_text, model="gpt", category="auto_cot", max_tokens=256, temperature=1.0, top_p=0.6) # Creative

----------------------------------------------------------------------------------------------------
GPT-4o-Mini - Zero Shot/Automated Chain of Thoughts (CoT) - T = 0.001 P = 0.95
----------------------------------------------------------------------------------------------------

To classify the article, let's analyze its content:

1. **Subject Matter**: The article discusses the release of "Madden NFL 2005," a football video game by EA Sports. It mentions the impact of the game's release on people's attendance at work, suggesting that some fans might take a sick day to play it.

2. **Context**: The focus is on a video game and its cultural impact, particularly in relation to sports (football) and gaming.

3. **Keywords**: The article includes terms like "Madden NFL," "EA Sports," and "football simulation," which are all related to sports and gaming.

4. **Category Options**: The categories provided are World, Sports, Business, and Sci/Tech. The article does not discuss global events 

In [ ]:
article_text = """
'Madden,' 'ESPN' Football Score in Different Ways (Reuters) Reuters - Was absenteeism a little high on Tuesday among the guys at the office? EA Sports would like to think it was because "Madden NFL 2005" came out that day, and some fans of the football simulation are rabid enough to take a sick day to play it.
"""
test_prompts(article_text, model="llama", category="cot", max_tokens=256, temperature=0.001, top_p=0.95) # Precise/Deterministic
test_prompts(article_text, model="llama", category="cot", max_tokens=256, temperature=0.6, top_p=0.8) # Balanced/Regular
test_prompts(article_text, model="llama", category="cot", max_tokens=256, temperature=1.0, top_p=0.6) # Creative

----------------------------------------------------------------------------------------------------
Llama - Chain of Thoughts (CoT) - T = 0.001 P = 0.95
----------------------------------------------------------------------------------------------------

 Sci/Tech

  Reasoning:
   1. The main topic of the article is the release of "Madden NFL 2005" and its impact on people's daily activities.
   2. The topic is related to the video game industry, which falls under the category of Sci/Tech.
   3. The category Sci/Tech is the most suitable for this article because it deals with the release of a video game and its effects on people's daily lives.
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Llama - Chain of Thoughts (CoT) - T = 0.6 P = 0.8
-----------------------------------------------------------------------------------------------

In [ ]:
test_prompts(article_text, model="gemini", category="cot", max_tokens=256, temperature=0.001, top_p=0.95) # Precise/Deterministic
test_prompts(article_text, model="gemini", category="cot", max_tokens=256, temperature=0.6, top_p=0.8) # Balanced/Regular
test_prompts(article_text, model="gemini", category="cot", max_tokens=256, temperature=1.0, top_p=0.6) # Creative

----------------------------------------------------------------------------------------------------
Gemini - Chain of Thoughts (CoT) - T = 0.001 P = 0.95
----------------------------------------------------------------------------------------------------

Category: **Sci/Tech**

**Reasoning:**

1. **Main topic:** The article focuses on the release of a video game, "Madden NFL 2005," and its impact on fans.
2. **Category:** Video games fall under the category of technology and entertainment.
3. **Assignment:**  Therefore, the article is classified as **Sci/Tech**. 

----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Gemini - Chain of Thoughts (CoT) - T = 0.6 P = 0.8
----------------------------------------------------------------------------------------------------

Category: **Sci/Tech**

**Reasoning:**

1. **Main topic:** The article f

In [ ]:
test_prompts(article_text, model="gpt", category="cot", max_tokens=256, temperature=0.0, top_p=0.95) # Precise/Deterministic
test_prompts(article_text, model="gpt", category="cot", max_tokens=256, temperature=0.6, top_p=0.8) # Balanced/Regular
test_prompts(article_text, model="gpt", category="cot", max_tokens=256, temperature=1.0, top_p=0.6) # Creative

----------------------------------------------------------------------------------------------------
GPT-4o-Mini - Chain of Thoughts (CoT) - T = 0.0 P = 0.95
----------------------------------------------------------------------------------------------------

Category: Sports
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
GPT-4o-Mini - Chain of Thoughts (CoT) - T = 0.6 P = 0.8
----------------------------------------------------------------------------------------------------

Category: Sports

Reasoning:
1. The main topic of the article revolves around the release of the video game "Madden NFL 2005" and its impact on fans, particularly in relation to football.
2. The topic fits into the Sports category as it discusses a football simulation game and its connection to the sport of football.
3. Therefore, the appropriate category for t

In [ ]:
article_text = """
'Madden,' 'ESPN' Football Score in Different Ways (Reuters) Reuters - Was absenteeism a little high on Tuesday among the guys at the office? EA Sports would like to think it was because "Madden NFL 2005" came out that day, and some fans of the football simulation are rabid enough to take a sick day to play it.
"""
test_prompts(article_text, model="llama", category="tot", max_tokens=512, temperature=0.001, top_p=0.95) # Precise/Deterministic
test_prompts(article_text, model="llama", category="tot", max_tokens=512, temperature=0.6, top_p=0.8) # Balanced/Regular
test_prompts(article_text, model="llama", category="tot", max_tokens=512, temperature=1.0, top_p=0.6) # Creative

----------------------------------------------------------------------------------------------------
Llama - Tree of Thoughts (ToT) - T = 0.001 P = 0.95
----------------------------------------------------------------------------------------------------

 Path 1: Identify the main topic of the article.
     The main topic of the article is the release of a new video game, "Madden NFL 2005".
     Path 2: Determine which category the topic fits into.
     The topic fits into the category of "Business" or "Sci/Tech" because it is about a new video game release.
     Path 3: Assign the category based on the topic.
     The category is "Sci/Tech" because it is about a new video game release, which is a technology-related topic.

    Final Answer:
    The category is "Sci/Tech".  All experts agree on this answer.  No expert leaves the group.  The group converges to the same answer.  The answer is correct.  The article is about the release of a new video game, which is a technology-related to

In [ ]:
test_prompts(article_text, model="gemini", category="tot", max_tokens=512, temperature=0.0, top_p=0.95) # Precise/Deterministic
test_prompts(article_text, model="gemini", category="tot", max_tokens=512, temperature=0.6, top_p=0.8) # Balanced/Regular
test_prompts(article_text, model="gemini", category="tot", max_tokens=512, temperature=1.0, top_p=0.6) # Creative

----------------------------------------------------------------------------------------------------
Gemini - Tree of Thoughts (ToT) - T = 0.0 P = 0.95
----------------------------------------------------------------------------------------------------

## Expert Reasoning:

**Path 1:**

**Expert 1:** The main topic of the article is the release of the video game "Madden NFL 2005" and its impact on fans.

**Expert 2:** Video games typically fall under the Sci/Tech category.

**Expert 3:**  Therefore, the article should be classified as **Sci/Tech**. 

**Path 2:**

**Expert 1:** Key phrases in the article include "Madden NFL 2005", "EA Sports", "football simulation", and "sick day to play it".

**Expert 2:**  "Football simulation" and "EA Sports" suggest a connection to sports.

**Expert 3:**  However, the focus on the game's release and impact on fans leans towards Sci/Tech. 

**Expert 2:**  I'm not sure. The article mentions both sports and technology. 

**Expert 2:**  I'm out. 

**Pa

In [ ]:
test_prompts(article_text, model="gpt", category="tot", max_tokens=512, temperature=0.0, top_p=0.95) # Precise/Deterministic
test_prompts(article_text, model="gpt", category="tot", max_tokens=512, temperature=0.6, top_p=0.8) # Balanced/Regular
test_prompts(article_text, model="gpt", category="tot", max_tokens=512, temperature=1.0, top_p=0.6) # Creative

----------------------------------------------------------------------------------------------------
GPT-4o-Mini - Tree of Thoughts (ToT) - T = 0.0 P = 0.95
----------------------------------------------------------------------------------------------------

To classify the article, let's follow the reasoning paths step by step.

**Path 1:**
1. Identify the main topic of the article.  
   - The main topic is about the release of "Madden NFL 2005" and its impact on fans, particularly regarding absenteeism at work.

**Path 2:**
1. Identify key phrases in the article.  
   - Key phrases include "Madden NFL 2005," "EA Sports," "football simulation," and "absenteeism."

**Path 3:**
1. Analyze the context and events described in the article.  
   - The article discusses a video game release related to football, which is a popular sport and entertainment topic.

Now, let's proceed to the next steps for each path.

**Path 1:**
2. Determine which category the topic fits into.  
   - The topic r

# Text Summarization

### Zero Shot Prompting

In [ ]:
def zero_shot(article_text):
    return f"""
Summarize the key points of the following news article in a concise paragraph:

{article_text}

Summary:
"""

### One Shot Prompting

In [ ]:
def one_shot(article_text):
    return f"""
Read the following news article and its summary, then provide an improved summary in a few sentences:

Example:
Article: (CNN) -- Football superstar, celebrity, fashion icon, multimillion-dollar heartthrob. Now, David Beckham is headed for the Hollywood Hills as he takes his game to U.S. Major League Soccer. CNN looks at how Bekham fulfilled his dream of playing for Manchester United, and his time playing for England. The world's famous footballer has begun a five-year contract with the Los Angeles Galaxy team, and on Friday Beckham will meet the press and reveal his new shirt number. This week, we take an in depth look at the life and times of Beckham, as CNN's very own "Becks," Becky Anderson, sets out to examine what makes the man tick -- as footballer, fashion icon and global phenomenon. It's a long way from the streets of east London to the Hollywood Hills and Becky charts Beckham's incredible rise to football stardom, a journey that has seen his skills grace the greatest stages in world soccer. She goes in pursuit of the current hottest property on the sports/celebrity circuit in the U.S. and along the way explores exactly what's behind the man with the golden boot. CNN will look back at the life of Beckham, the wonderfully talented youngster who fulfilled his dream of playing for Manchester United, his marriage to pop star Victoria, and the trials and tribulations of playing for England. We'll look at the highs (scoring against Greece), the lows (being sent off during the World Cup), the Man. U departure for the Galacticos of Madrid -- and now the Home Depot stadium in L.A. We'll ask how Beckham and his family will adapt to life in Los Angeles -- the people, the places to see and be seen and the celebrity endorsement. Beckham is no stranger to exposure. He has teamed with Reggie Bush in an Adidas commercial, is the face of Motorola, is the face on a PlayStation game and doesn't need fashion tips as he has his own international clothing line. But what does the star couple need to do to become an accepted part of Tinseltown's glitterati? The road to major league football in the U.S.A. is a well-worn route for some of the world's greatest players. We talk to some of the former greats who came before him and examine what impact these overseas stars had on U.S. soccer and look at what is different now. We also get a rare glimpse inside the David Beckham academy in L.A, find out what drives the kids and who are their heroes. The perception that in the U.S.A. soccer is a "game for girls" after the teenage years is changing. More and more young kids are choosing the European game over the traditional U.S. sports. E-mail to a friend .

Summary: Beckham has agreed to a five-year contract with Los Angeles Galaxy . New contract took effect July 1, 2007 . Former English captain to meet press, unveil new shirt number Friday . CNN to look at Beckham as footballer, fashion icon and global phenomenon .

Article: {article_text}

Summary:
"""

### Few Shot Prompting

In [ ]:
def few_shot(article_text):
    return f"""
Read the following news articles and their summaries, then provide a summary of the next article that focuses on the key events and their impact:

Examples:
Article 1: (CNN) -- They wore feathers. They wore fancy hats. And of course, they wore fur. Never mind the human -- the dog in haute couture was the fashionistas' focus during Pet Fashion Week. But the models strutting down the runway were of the four-legged variety. The glamorous pooches were accompanied by human models -- but the furry ones were getting all the attention. It was Pet Fashion Week New York and these canines were not wearing the boring plaid raincoats that have sold for years. They were wearing one-of-kind design creations. The show last weekend was aimed at owners of sophisticated canines who may be willing to pay for their pup's own stylist. Booths at the annual event features couture clothing, jewelry and other accessories for the well-dressed doggie -- all part of the $40 billion pet industry. E-mail to a friend .
Summary 1: Dogs ruled in New York during Pet Fashion Week . Dazzling couture designs trotted down the runway . Upscale pet accessories, apparel, and lifestyle items on display . Design awards encourage pushing the envelope in style .

Article 2: JOHANNESBURG, South Africa -- South African fast bowler Dale Steyn took a career-best five for 34 as the Proteas took a tight grip on the first test against New Zealand in Johannesburg. Steyn's career-best 5-34 was his fourth five-wicket haul in 14 tests. New Zealand were bowled out for 118 in reply to South Africa's 226 and the home side piled on the agony by reaching 179 for two in their second innings. Hashim Amla and Jacques Kallis shared an unbeaten stand of 159 as South Africa stretched their lead to 287. South Africa's bowlers excelled to bring their side back into the game after their disappointing first innings. They snapped up five wickets in the morning session when the Kiwis could only muster 56 runs. Former New Zealand captain Stephen Fleming made 40 but the next best score was new cap Ross Taylor's 15. Fleming was struck on the right forearm by Steyn and did not field during the afternoon. Coach John Bracewell said he had gone for precautionary X-rays but there was only bruising. New Zealand, 41 for two overnight, lost nightwatchman Shane Bond, bowled by a Steyn yorker, before Makhaya Ntini claimed the crucial wicket of Fleming, who was well caught by AB de Villiers diving to his left at third slip. Scott Styris and Taylor scraped 19 runs in 10 overs before more wickets tumbled. Steyn's figures bettered his previous best of five for 47 against the same opponents at Centurion two seasons ago. It was his fourth five-wicket haul in 14 tests. Ntini took three for 47 and Kallis two for 11. South Africa made an uncertain start to their second innings with openers Herschelle Gibbs and captain Graeme Smith out cheaply, but Amla and Kallis blunted the attack and then took charge. They batted together for 205 minutes, Amla facing 230 balls and hitting 13 boundaries in his 85 while Kallis hit 12 fours off 122 deliveries in reaching 76. The Kiwis were left to regret Brendon McCullum's failure to hold a chance from Amla off Shane Bond, when the batsman had only scored two. "The ball was hard and new and we were trying to get momentum. It cost us a lot," said coach John Bracewell. E-mail to a friend .
Summary 2: South Africa lead New Zealand by 287 with 8 wickets standing in the 1st test . The Proteas reach 179-2 in their second innings after the Kiwis are 118 all out . South African paceman Dale Steyn takes a career-best 5-34.

Article 3:{article_text}
Summary 3:
"""

### Zero Shot/Automated Chain of Thoughts (CoT)

In [ ]:
def auto_cot(article_text):
    return f"""
Summarize the key points of the following news article in a concise paragraph:

Article: {article_text}

Summary:

Let's think step-by-step.
"""

### Chain of Thoughts (CoT) prompting

In [ ]:
def cot(article_text):
  return f"""
  Analyze the structure and key points of the following news article, then provide a summary that captures the main ideas. Think through each step of your analysis:

  {article_text}

  Step 1: Identify the main topic and central narrative of the article.

  Step 2: Outline the key events or developments discussed in the article.

  Step 3: Analyze how the article is structured (e.g., chronological, problem-solution, cause-effect).

  Step 4: Determine the most important information to include in the summary.

  Summary:
  """

### Tree of Thoughts (ToT) prompting

In [ ]:
def tot(article_text):
    return f"""
    Analyze the following news article from multiple perspectives, then provide a summary that captures the diverse viewpoints. Explore different reasoning paths:

    {article_text}

    Path 1 - Government/Policymaker Perspective:
    - Key points from this perspective:
    - Reasoning for this perspective:
    - Pros and cons of this perspective:

    Path 2 - Business/Industry/Sports/Celebrity Perspective:
    - Key points from this perspective:
    - Reasoning for this perspective:
    - Pros and cons of this perspective:

    Path 3 - Citizen/Consumer Perspective:
    - Key points from this perspective:
    - Reasoning for this perspective:
    - Pros and cons of this perspective:

    Evaluate the different perspectives:
    - Most important perspectives to include in the summary:
    - Reasoning for this selection:

    Summary:
    """

In [ ]:
article_text = """Australia leg-spinner Stuart MacGill has announced he will quit international cricket at the end of the ongoing second Test against West Indies. MacGill will retire after 10 years of Test cricket, in which he has taken 207 wickets. The 37-year-old made his Test debut against South Africa 10 years ago and has since gone on to take 207 wickets at an average of 28.28 over 43 Test matches. "Unfortunately now my time is up," MacGill said. "I am incredibly lucky that as well as providing me with amazing opportunities off the field, my job allows me to test myself in one of Australia's most highly scrutinised sporting environments. "Bowling with some of crickets all time greats such as Glenn McGrath, Shane Warne, Jason Gillespie and Brett Lee has made my job a lot easier. I want to be sure that exciting young bowlers like Mitchell Johnson enjoy the same privilege," he added. MacGill took the only wicket to fall on a rain-interrupted third day of the Test in Antigua. He had Ramnaresh Sarwan brilliantly caught at slip by Michael Clarke for a well-constructed 65, but otherwise drew blank on a frustrating day for the tourists. The ever dependable Shivnarine Chanderpaul (55 not out) and Dwayne Bravo (29 not out) took the West Indies to the close on 255 for four wickets. They were replying to Australia's 479 for seven declared and with only two days remaining a draw looks the likely outcome in MacGill's farewell appearance. Australia won the first Test in Jamaica by 95 runs."""
gold_summary = "Australian leg-spinner Stuart MacGill has announced he will quit Test cricket . The 37-year-old made his Test debut 10 years ago and has taken 207 wickets . MacGill took only wicket to fall in rain-interrupted third day of second Test . Shivnarine Chanderpaul and Dwayne Bravo compile unbroken stand of 73 ."
test_prompts(article_text, reference_summary=gold_summary, model="llama", category="zero_shot", max_tokens=128, temperature=0.001, top_p=0.95) # Precise/Deterministic
test_prompts(article_text, reference_summary=gold_summary, model="llama", category="zero_shot", max_tokens=128, temperature=0.6, top_p=0.8) # Balanced/Regular
test_prompts(article_text, reference_summary=gold_summary, model="llama", category="zero_shot", max_tokens=128, temperature=1.0, top_p=0.6) # Creative

----------------------------------------------------------------------------------------------------
Llama - Zero Shot - T = 0.001 P = 0.95
----------------------------------------------------------------------------------------------------

Stuart MacGill, a 37-year-old Australian leg-spinner, has announced his retirement from international cricket after the ongoing second Test against West Indies. With 207 wickets in 43 Test matches, MacGill has had a successful 10-year career, during which he has played alongside some of cricket's greatest bowlers. He will retire after the current Test, which is likely to end in a draw, and has expressed his gratitude for the opportunities he has had to test himself in a highly competitive environment. MacGill's farewell appearance will be his last in international cricket.
Rouge Score: {'rouge-1': {'r': 0.42857142857142855, 'p': 0.26865671641791045, 'f': 0.3302752246208232}, 'rouge-2': {'r': 0.10638297872340426, 'p': 0.05952380952380952, 'f': 0.076

In [ ]:
article_text = """Australia leg-spinner Stuart MacGill has announced he will quit international cricket at the end of the ongoing second Test against West Indies. MacGill will retire after 10 years of Test cricket, in which he has taken 207 wickets. The 37-year-old made his Test debut against South Africa 10 years ago and has since gone on to take 207 wickets at an average of 28.28 over 43 Test matches. "Unfortunately now my time is up," MacGill said. "I am incredibly lucky that as well as providing me with amazing opportunities off the field, my job allows me to test myself in one of Australia's most highly scrutinised sporting environments. "Bowling with some of crickets all time greats such as Glenn McGrath, Shane Warne, Jason Gillespie and Brett Lee has made my job a lot easier. I want to be sure that exciting young bowlers like Mitchell Johnson enjoy the same privilege," he added. MacGill took the only wicket to fall on a rain-interrupted third day of the Test in Antigua. He had Ramnaresh Sarwan brilliantly caught at slip by Michael Clarke for a well-constructed 65, but otherwise drew blank on a frustrating day for the tourists. The ever dependable Shivnarine Chanderpaul (55 not out) and Dwayne Bravo (29 not out) took the West Indies to the close on 255 for four wickets. They were replying to Australia's 479 for seven declared and with only two days remaining a draw looks the likely outcome in MacGill's farewell appearance. Australia won the first Test in Jamaica by 95 runs."""
gold_summary = "Australian leg-spinner Stuart MacGill has announced he will quit Test cricket . The 37-year-old made his Test debut 10 years ago and has taken 207 wickets . MacGill took only wicket to fall in rain-interrupted third day of second Test . Shivnarine Chanderpaul and Dwayne Bravo compile unbroken stand of 73 ."
test_prompts(article_text, reference_summary=gold_summary, model="gemini", category="zero_shot", max_tokens=128, temperature=0.001, top_p=0.95) # Precise/Deterministic
test_prompts(article_text, reference_summary=gold_summary, model="gemini", category="zero_shot", max_tokens=128, temperature=0.6, top_p=0.8) # Balanced/Regular
test_prompts(article_text, reference_summary=gold_summary, model="gemini", category="zero_shot", max_tokens=128, temperature=1.0, top_p=0.6) # Creative

----------------------------------------------------------------------------------------------------
Gemini - Zero Shot - T = 0.001 P = 0.95
----------------------------------------------------------------------------------------------------

Australian leg-spinner Stuart MacGill has announced his retirement from international cricket after 10 years and 207 wickets. The 37-year-old, who made his Test debut in 2001, will play his final match in the second Test against West Indies. MacGill expressed gratitude for the opportunities cricket has provided him and acknowledged the support of his fellow bowlers. He took one wicket on the rain-affected third day of the Test, but a draw seems likely in his farewell appearance. 

Rouge Score: {'rouge-1': {'r': 0.5952380952380952, 'p': 0.4032258064516129, 'f': 0.480769225954142}, 'rouge-2': {'r': 0.2765957446808511, 'p': 0.16883116883116883, 'f': 0.20967741464750272}, 'rouge-l': {'r': 0.5952380952380952, 'p': 0.4032258064516129, 'f': 0.48076922595

In [ ]:
article_text = """Australia leg-spinner Stuart MacGill has announced he will quit international cricket at the end of the ongoing second Test against West Indies. MacGill will retire after 10 years of Test cricket, in which he has taken 207 wickets. The 37-year-old made his Test debut against South Africa 10 years ago and has since gone on to take 207 wickets at an average of 28.28 over 43 Test matches. "Unfortunately now my time is up," MacGill said. "I am incredibly lucky that as well as providing me with amazing opportunities off the field, my job allows me to test myself in one of Australia's most highly scrutinised sporting environments. "Bowling with some of crickets all time greats such as Glenn McGrath, Shane Warne, Jason Gillespie and Brett Lee has made my job a lot easier. I want to be sure that exciting young bowlers like Mitchell Johnson enjoy the same privilege," he added. MacGill took the only wicket to fall on a rain-interrupted third day of the Test in Antigua. He had Ramnaresh Sarwan brilliantly caught at slip by Michael Clarke for a well-constructed 65, but otherwise drew blank on a frustrating day for the tourists. The ever dependable Shivnarine Chanderpaul (55 not out) and Dwayne Bravo (29 not out) took the West Indies to the close on 255 for four wickets. They were replying to Australia's 479 for seven declared and with only two days remaining a draw looks the likely outcome in MacGill's farewell appearance. Australia won the first Test in Jamaica by 95 runs."""
gold_summary = "Australian leg-spinner Stuart MacGill has announced he will quit Test cricket . The 37-year-old made his Test debut 10 years ago and has taken 207 wickets . MacGill took only wicket to fall in rain-interrupted third day of second Test . Shivnarine Chanderpaul and Dwayne Bravo compile unbroken stand of 73 ."
test_prompts(article_text, reference_summary=gold_summary, model="gpt", category="zero_shot", max_tokens=128, temperature=0.001, top_p=0.95) # Precise/Deterministic
test_prompts(article_text, reference_summary=gold_summary, model="gpt", category="zero_shot", max_tokens=128, temperature=0.6, top_p=0.8) # Balanced/Regular
test_prompts(article_text, reference_summary=gold_summary, model="gpt", category="zero_shot", max_tokens=128, temperature=1.0, top_p=0.6) # Creative

----------------------------------------------------------------------------------------------------
GPT-4o-Mini - Zero Shot - T = 0.001 P = 0.95
----------------------------------------------------------------------------------------------------

Australia leg-spinner Stuart MacGill has announced his retirement from international cricket following the ongoing second Test against the West Indies, concluding a decade-long career in which he took 207 wickets in 43 Test matches. The 37-year-old, who made his debut against South Africa, expressed gratitude for the opportunities his career provided and highlighted the privilege of bowling alongside legends like Glenn McGrath and Shane Warne. In his final match, MacGill took one wicket on a rain-affected day, but the match is likely to end in a draw as the West Indies reached 255 for four in response to Australia's 479 for seven declared. Australia had previously won the first Test by 95 runs.
Rouge Score: {'rouge-1': {'r': 0.5, 'p': 0.24137